# Búsqueda hiperparámetros

### Grupo 2:
     - Lucía Bouza  C.I 42897970
     - Juan Machado C.I 46919180
     



Para la búsqueda de parámetros de la red se utilizó RandomSearch con Keras Tuner. 
Se realizo el módulo TunerEsOdio, para esta sección. La búsqueda se realizó en función de:
- cantidad de unidades de la capa densa intermedia
- learning rate del método de optimización ADAM
- función de activación de la capa densa intermedia
- porcentaje de dropout de la capa de dropout

Los resultados de cada corrida quedan grabados, pero no se realizará la entrega de ellos ya 
que la carpeta sobrepasa los 2 GB. Si se desea correr el Tuneo de parámetros puede ejecutarse dicho script o este notebook. 

Los mejores hiperparámetros hallados fueron los siguientes, obteniendo accuracy de 71%:

96 Unidades en la primer capa densa
relu como función de activación en dicha capa
0.1 como porcentaje de dropout
0.0002965488058019691 como learning rate de ADAM

In [3]:
from kerastuner import HyperModel
from  keras import layers, optimizers
from keras.models import Sequential
from kerastuner.tuners import RandomSearch
import preprocesamiento
import sys


class HyperModel(HyperModel):
    def __init__(self, vocab_size, embedding_dim, input_length, embedding_matrix):
        self.vocab_size =  vocab_size
        self.embedding_dim =  embedding_dim
        self.input_length =  input_length
        self.embedding_matrix =  embedding_matrix

    def build(self, hp):
        model = Sequential()
        
        model.add(layers.Embedding(
            input_dim= self.vocab_size, 
            output_dim= self.embedding_dim, 
            input_length=100, 
            weights=[self.embedding_matrix], 
            trainable=True))
    
        model.add(layers.GlobalMaxPooling1D())
        
        model.add(
            layers.Dense(units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        model.add(
            layers.Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        
        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(
            optimizer= optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model

##################################################
######Lectura de parámetros de entrada###########
##################################################
cantParametros = len(sys.argv)
ArrayParametros = str(sys.argv)
#dataPath = ArrayParametros[0]
dataPath = './recursos_lab/'


##################################################
######Carga y Preprocesamiento de datos###########
##################################################
# Constantes
embedding_dim = 300
"En X_train tendremos la vectorización de la frase, y en Y_train la salida"
X_train, y_train, vocab_size, embedding_matrix, tokenizer, X_val, y_val = preprocesamiento.preprocesamientoTrain(dataPath, embedding_dim)

hypermodel = HyperModel(vocab_size, embedding_dim, X_train.shape[0], embedding_matrix)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=1,
    max_trials=1,
    executions_per_trial=1,
    directory='random_search',
    project_name='EsOdio'
)

tuner.search_space_summary()

tuner.search(X_train, y_train, epochs=50, verbose=False, validation_data=(X_val, y_val), batch_size=250, use_multiprocessing=True)

# Show a summary of the search
tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters()[0]
best_model.summary()

units = best_hp.get("units")
dense_activation = best_hp.get("dense_activation")
dropout_2 = best_hp.get("dropout_2")
learning_rate = best_hp.get("learning_rate")
print("units: ", units)
print("activación: ", dense_activation)
print("dropout: ",dropout_2)
print("learning rate: ", learning_rate)

# Evaluate the best model.
loss, accuracy = best_model.evaluate(X_val, y_val)

INFO:tensorflow:Oracle triggered exit


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          3902700   
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 96)                28896     
_________________________________________________________________
dropout (Dropout)            (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 97        
Total params: 3,931,693
Trainable params: 3,931,693
Non-trainable params: 0
_________________________________________________________________
units:  96
activación:  relu
dropout:  0.1
learning rate:  0.0002965488058019691
17/17 [========================